In [1]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import MultiLineString

In [2]:
stations_dir = "../Data/stations.csv"
lines_def_dir = "../Data/lines_definition.csv"
route_dir = "../Data/routes.csv"

In [3]:
tube_df = pd.read_csv(stations_dir, index_col='id') #this file contains the name, zone, total lines and rail per station, also the zone it belongs in the metro system
lines_df = pd.read_csv(lines_def_dir) #this file contains the conexions between each station
routes_df = pd.read_csv(route_dir) #this file contains the info of each metro line, the number, the color and the stripe if it has

In the next cell I create a GeoDataFrame with the latitude and longitude as the geomatry column in the form of a shapely Point 

In [4]:
tube_df['geometry'] = None
for index, row in tube_df.iterrows():
    tube_df.loc[index, ['geometry']] = Point(row['latitude'], row['longitude'])
tube_df.pop('latitude')
tube_df.pop('longitude') #there's no need to keep doubleled data in the dataframe
tube_df = gpd.GeoDataFrame(tube_df)

In the next cell we create a LineString for each conection between stations, so that later we can show the metro lsystem on the map

In [5]:
lines_df['geometry'] = None

for index, row in lines_df.iterrows(): 
    lines_df.loc[index, ['geometry']] = LineString([tube_df.loc[row.station1, ['geometry']].values[0], tube_df.loc[row.station2, ['geometry']].values[0]])

A little messy, baut in this next cell we concatenate each LineString in a MultilineString, just in case, to have all those conections in the routes geometry column

In [6]:
routes_df['geometry'] = None

for index, row in routes_df.iterrows():
    lines_tube = lines_df.loc[lines_df['line'] == row["line"], ['geometry']].values.tolist()
    lines_aux = []
    for n in lines_tube:
        lines_aux.append(n.pop())
    
    routes_df.loc[index, ['geometry']] = MultiLineString(lines_aux)

Now we insert each line in a map with its colour and the stations in the form of circles with the zone it belongs represented by a colour

In [7]:
def zone_color(zone):
    zones = ['#006400' ,'#008000' ,'#00FF00' , #dark_green,  green,         light green
             '#FFFF00' ,'#FFA500' ,'#FF4500' , #yellow,      orange-yellow, orange
             '#FF3300' ,'#FF0000' ,'#8B0000']  #dark orange, light red,     dark red
    return zones[zone-1]



m_1 = folium.Map(location = [51.5074, -0.1278])

for index, row in routes_df.iterrows():
    colour = row.colour
    for line in row.geometry.geoms:
        folium.PolyLine(locations = list(line.coords), color = "#" + colour).add_to(m_1)
    
for index, row in tube_df.iterrows():
    folium.Circle(location = row.geometry.coords[0],radius=5, color = zone_color(int(tube_df.loc[index, ['zone']].values.tolist()[0]))).add_to(m_1)

In [ ]:
#Uncomment to show teh map on your browser
#m_1.show_in_browser()